In [4]:
from transformers import AutoModel, AutoTokenizer
import torch

# Путь к сохранённой модели
MODEL_DIR = "saved_e5_model"

# Загрузка токенизатора и модели
tokenizer = AutoTokenizer.from_pretrained(MODEL_DIR)
model = AutoModel.from_pretrained(MODEL_DIR)
model.eval()  # Обязательно для инференса
device = torch.device('cuda:6' if torch.cuda.is_available() else 'cpu')
model = model.to(device)


/home/sirius3085/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
def encode_texts(texts, prefix="query", batch_size=32):
    """
    Кодирует список текстов в эмбеддинги (используется [CLS] токен).
    prefix: "query" или "passage" (для правильного шаблона).
    """
    embeddings = []

    for i in range(0, len(texts), batch_size):
        batch = texts[i:i+batch_size]
        inputs = tokenizer(
            [f"{prefix}: {text}" for text in batch],
            padding=True,
            truncation=True,
            return_tensors='pt'
        ).to(device)

        with torch.no_grad():
            output = model(**inputs)
            cls_emb = output.last_hidden_state[:, 0]  # [CLS] токен
            embeddings.append(cls_emb.cpu())

    return torch.cat(embeddings, dim=0).numpy()

In [6]:
# Пример запроса
queries = ["wireless bluetooth headphones", "usb-c charging cable"]
query_embs = encode_texts(queries, prefix="query")

print("✅ Эмбеддинги запросов:", query_embs.shape)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


✅ Эмбеддинги запросов: (2, 384)


In [7]:
# --- 1. Загрузка исходных данных ---
from datasets import load_dataset
import pandas as pd
from tqdm import tqdm
import os
import numpy as np

PRODUCTS_PATH = "product_titles.csv"
EMBEDS_PATH = "product_embeddings.npy"
# EMBEDS_PATH = "small_product_embeddings.npy"
MODEL_PATH = "saved_e5_model"

In [12]:
# --- 1. Загрузка товаров ---
if os.path.exists(PRODUCTS_PATH):
    print("📦 Загружаем сохранённые товары и эмбеддинги...")
    product_titles = pd.read_csv(PRODUCTS_PATH).squeeze().tolist()
    print(f"✅ Загрузили {len(product_titles)} товаров")
else:
    print("📥 Загружаем датасет с Hugging Face...")
    dataset = load_dataset("tasksource/esci", split="train")
    df = pd.DataFrame([x for x in tqdm(dataset, desc="📄 Преобразуем в DataFrame")])

    product_titles = df['product_title'].dropna().unique().tolist()
    print(f"🛍️ Уникальных товаров: {len(product_titles)}")
    pd.Series(product_titles).to_csv(PRODUCTS_PATH, index=False)

📥 Загружаем датасет с Hugging Face...


📄 Преобразуем в DataFrame: 100%|██████████| 2027874/2027874 [03:59<00:00, 8483.66it/s]


🛍️ Уникальных товаров: 1423918


In [15]:
def encode_texts(texts, prefix="passage", batch_size=64):
    all_embeddings = []
    for i in tqdm(range(0, len(texts), batch_size), desc="🚀 Кодируем товары"):
        batch = texts[i:i+batch_size]
        inputs = tokenizer([f"{prefix}: {t}" for t in batch], padding=True, truncation=True, return_tensors='pt')
        with torch.no_grad():
            outputs = model(
                input_ids=inputs['input_ids'].cuda(),
                attention_mask=inputs['attention_mask'].cuda()
            )
        emb = outputs.last_hidden_state[:, 0].cpu().numpy()
        all_embeddings.append(emb)
    return np.vstack(all_embeddings)

In [26]:
# Загрузка модели
print("📦 Загружаем сохранённую модель...")
model = AutoModel.from_pretrained(MODEL_PATH).eval().cuda()
tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)

# Кодирование и сохранение
product_embs = encode_texts(product_titles, batch_size=128)
np.save(EMBEDS_PATH, product_embs)
print("✅ Эмбеддинги и названия товаров сохранены.")

📦 Загружаем сохранённую модель...


🚀 Кодируем товары: 100%|██████████| 11125/11125 [09:13<00:00, 20.10it/s]


✅ Эмбеддинги и названия товаров сохранены.


In [38]:
_product_title = "Wireless Headset"
_query = "wireless headphones"

# Кодируем с нужными префиксами
t = encode_texts([_product_title], prefix="passage")  # → (1, dim)
q = encode_texts([_query], prefix="query")            # → (1, dim)

# Нормализация
t = t / np.linalg.norm(t, axis=1, keepdims=True)
q = q / np.linalg.norm(q, axis=1, keepdims=True)

# Скаларное произведение → косинусная близость
score = np.dot(q, t.T)[0][0]

print(f"📊 Similarity score: {score:.4f}")
# +- совпадает с продом!

🚀 Кодируем товары: 100%|██████████| 1/1 [00:00<00:00, 67.14it/s]

📊 Similarity score: 0.9837


In [8]:
import pandas as pd
import os

# добавляем статистику показов для каждого товара
# ——— Загружаем датасет, если еще не загружен ———
if not os.path.exists(PRODUCTS_PATH) or not os.path.exists("product_stats.csv"):
    print("📥 Загружаем датасет с Hugging Face...")
    dataset = load_dataset("tasksource/esci", split="train")
    df = pd.DataFrame([x for x in tqdm(dataset, desc="📄 Преобразуем в DataFrame")])
    
    # Уникальные товары
    product_titles = df['product_title'].dropna().unique().tolist()
    pd.Series(product_titles).to_csv(PRODUCTS_PATH, index=False)

    # ——— Считаем количество показов каждого товара ———
    print("📊 Считаем статистику по товарам...")
    stats = df['product_title'].value_counts().reset_index()
    stats.columns = ['product_title', 'views']
    stats.to_csv("product_stats.csv", index=False)
    print(f"✅ Сохранили статистику для {len(stats)} товаров")

📥 Загружаем датасет с Hugging Face...


📄 Преобразуем в DataFrame: 100%|██████████| 2027874/2027874 [04:00<00:00, 8425.57it/s]


📊 Считаем статистику по товарам...
✅ Сохранили статистику для 1423918 товаров


In [11]:
_stats = pd.read_csv("product_stats.csv", index_col=0)

In [12]:
_stats.head()

,views
product_title,
Pilot,161
Episode 1,157
Currently Unsupervised Novelty Graphic Sarcastic Funny T Shirt XL Black,92
変な家,88
Acer (エイサー) Aspire 5 スリム ノートパソコン 15.6インチ フルHD IPS ディスプレイ AMD(アドバンスト・マイクロ・デバイセズ) Ryzen(ライゼン) 3 3200U (日本語配列ではない場合があります),85
